```
This notebook sets up and runs a test case for analyzing Kelvin waves
Copyright (C) 2018 - 2022 SINTEF Digital
Copyright (C) 2018 - 2022 Norwegian Meteorological Institute

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
import sys
gpuocean_path = [p[:-4] for p in sys.path if p.endswith("gpuocean/src")][0]
import git
repo = git.Repo(gpuocean_path)
print("GPUOcean code from:", repo.head.object.hexsha, "on branch", repo.active_branch.name)

# Oslofjord
Testing of Nils projected files

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime, copy
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

In [ ]:
from gpuocean.utils import IPythonMagic, NetCDFInitialization

In [ ]:
%cuda_context_handler gpu_ctx

Path to the test file

In [ ]:
source_url = "/sintef/data/OsloFjord/test_polstere.nc"

## Calculating MLD

In [ ]:
nc = Dataset(source_url)
dimY, dimX = nc.dimensions["eta_rho"].size, nc.dimensions["xi_rho"].size

In [ ]:
x0, x1, y0, y1 = 5, dimX-5, 135, dimY-5

In [ ]:
mld = NetCDFInitialization.MLD(source_url, 1023, min_mld=5, max_mld=40, x0=x0, x1=x1, y0=y0, y1=y1)

Smoothing MLD

In [ ]:
s_nc = Dataset(source_url)
s_hs = s_nc["h"][y0:y1,x0:x1]

bad_yx = np.where(np.logical_or(np.abs(mld - s_hs) < 0.35*s_hs, np.abs(mld - s_hs) < 3))
bad_mask = np.where(np.logical_and((s_hs!=0.0), np.logical_and(np.abs(mld - s_hs) > 0.35*s_hs, np.abs(mld - s_hs) > 3)),0,1)

Xidx = np.arange(0, mld.shape[1])
Yidx = np.arange(0, mld.shape[0])

xx, yy = np.meshgrid(Xidx, Yidx)

K = 20

for i in range(len(bad_yx[0])):
    dists = (xx-bad_yx[1][i])**2 + (yy-bad_yx[0][i])**2 + 1e5*bad_mask
    sum = 0.0
    for k in range(K): 
        sum += mld[np.unravel_index(dists.argmin(), dists.shape)]
        dists[np.unravel_index(dists.argmin(), dists.shape)] = 1e5
    mld[bad_yx[0][i],bad_yx[1][i]] = sum/K

In [ ]:
mld = NetCDFInitialization.fill_coastal_data(mld)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,10))

im = axs[0].imshow(np.ma.array(nc["h"][y0:y1,x0:x1], mask=(nc["h"][y0:y1,x0:x1]==0)), cmap="cool")
plt.colorbar(im, ax=axs[0], shrink=0.5)
axs[0].set_title("Bathymetry")

im = axs[1].imshow(mld, cmap="cool", vmin=0.0, vmax=10)
plt.colorbar(im, ax=axs[1], shrink=0.5)
axs[1].set_title("MLD")

im = axs[2].imshow(bad_mask-(s_hs==0), cmap="Reds")
plt.colorbar(im, ax=axs[2], shrink=0.5)
axs[2].set_title("Bad values")



In [ ]:
y_cut = 120 # See aboves plots, vertical north-south along this index!

s_pot_densities = NetCDFInitialization.potentialDensities(source_url, t=0, x0=x0, x1=x1, y0=y0, y1=y1)

s_nc = Dataset(source_url)
s_hs   = s_nc["h"][y0:y1,x0:x1]
s_rhos = s_nc["Cs_r"][:]

s_pot_densities_show = np.ma.array(np.zeros((s_pot_densities.shape[1],100)))
for l in range(s_pot_densities.shape[1]):
    d_up = 0 
    for i in reversed(range(len(s_rhos))):
        d = round(-(s_hs[l,y_cut]*s_rhos[i]))
        s_pot_densities_show[l,d_up:d] = s_pot_densities[i][l,y_cut]
        d_up = d
s_pot_densities_show.mask = (s_pot_densities_show==0)

plt.figure(figsize=(15,5))
plt.imshow(s_pot_densities_show.T[0:50], origin="upper", cmap="plasma",  aspect='auto')
plt.colorbar(label="Potential Density [kg/m^3]")
plt.scatter(np.arange(s_pot_densities.shape[1]), mld[:,y_cut], c=np.where(bad_mask[:,y_cut]==1,'g', 'b'))

plt.suptitle("Mixed Layer Depth")
plt.ylabel("depth [m]")
plt.xlabel("blue dots are directly calcluted MLD, green dots are smoothed")
plt.tight_layout()